# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
# %pip install duckdb
# %pip install jupysql
# %pip install duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

In [3]:
dataset_path = 'C:/Users/pedro/Documents/DMEyF/dmeyf2025/data/'
dataset_file = 'processed/competencia_01.csv'

data = pd.read_csv(dataset_path + dataset_file)

In [4]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [5]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor
0,-302,-1763,-302,-1763
1,-274,-1735,-274,-1735
2,-243,-1704,-243,-1704
3,-213,-1674,-213,-1674
4,-182,-1643,-182,-1643
5,-152,-1613,-152,-1613
6,-757,-1032,-757,-1032
7,-729,-1004,-729,-1004
8,-698,-973,-698,-973
9,-668,-943,-668,-943


Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [6]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,21720.62,19216.74,40937.36
1,53966.31,34021.47,87987.78
2,87993.72,39771.39,127765.11
3,43647.07,21194.22,64841.29
4,18977.29,47900.09,66877.38
5,12722.85,174114.81,186837.66
6,434.01,11989.48,12423.49
7,868.02,16169.05,17037.07
8,0.00,16172.29,16172.29
9,434.01,16174.44,16608.45


Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [7]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,135524.48,NaN
1,NaN,158113.39,NaN
2,NaN,148122.11,NaN
3,NaN,177459.18,NaN
4,NaN,177972.42,NaN
5,NaN,170233.70,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


Esto no siempre es deseable y puede ser fácilmente evitable

In [8]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,21720.62,19216.74,40937.36
1,53966.31,34021.47,87987.78
2,87993.72,39771.39,127765.11
3,43647.07,21194.22,64841.29
4,18977.29,47900.09,66877.38
5,12722.85,174114.81,186837.66
6,434.01,11989.48,12423.49
7,868.02,16169.05,17037.07
8,0.00,16172.29,16172.29
9,434.01,16174.44,16608.45


In [9]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);


,Success


In [10]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,124770.71,124770.71
1,NaN,102486.37,102486.37
2,NaN,19592.71,19592.71
3,NaN,2933.91,2933.91
4,NaN,13286.55,13286.55
5,NaN,22405.78,22405.78
6,NaN,3365.32,3365.32
7,NaN,59404.38,59404.38
8,NaN,18395.03,18395.03
9,NaN,52468.95,52468.95


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [11]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


,foto_mes,numero_de_cliente,cliente_antiguedad,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,202101,249221323,93,1,0.379128,0.382611,2,4
1,202102,249221323,94,2,0.381582,0.385027,2,4
2,202103,249221323,95,3,0.384825,0.388262,2,4
3,202104,249221323,96,4,0.387151,0.390593,2,4
4,202105,249221323,97,5,0.389944,0.393394,2,4
...,...,...,...,...,...,...,...,...
978434,202106,1598368433,1,1,0.000000,0.001808,1,1
978435,202106,1598419415,1,1,0.000000,0.001808,1,1
978436,202106,1598425905,1,1,0.000000,0.001808,1,1
978437,202106,1598470389,1,1,0.000000,0.001808,1,1


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [12]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,482604778,202103,111,111
1,482604778,202104,100,111
2,482604778,202105,93,100
3,482604778,202106,91,93
4,482696811,202101,55,<NA>
5,482696811,202102,59,55
6,482696811,202103,53,59
7,482696811,202104,62,53
8,482696811,202105,65,62
9,482696811,202106,73,65


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [13]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,643929649,202102,36,23,13,<NA>
1,643929649,202103,37,36,1,14
2,643929649,202104,32,37,-5,-4
3,643929649,202105,27,32,-5,-10
4,643929649,202106,32,27,5,0
5,644055892,202101,54,<NA>,<NA>,<NA>
6,644055892,202102,46,54,-8,<NA>
7,644055892,202103,40,46,-6,-14
8,644055892,202104,46,40,6,0
9,644055892,202105,54,46,8,14


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [14]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,lag_2_ctrx_quarter,lag_3_ctrx_quarter,avg_3_ctrx_quarter
0,249221323,202106,91,81,89,89,87.500000
1,249221323,202105,81,89,89,97,89.000000
2,249221323,202104,89,89,97,82,89.250000
3,249221323,202103,89,97,82,<NA>,89.333333
4,249221323,202102,97,82,<NA>,<NA>,89.500000
5,249221323,202101,82,<NA>,<NA>,<NA>,82.000000
6,249227600,202106,150,170,155,147,155.500000
7,249227600,202105,170,155,147,126,149.500000
8,249227600,202104,155,147,126,148,144.000000
9,249227600,202103,147,126,148,<NA>,140.333333


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [15]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_3,ctrx_quarter_max_3,ctrx_quarter_min_3
0,249274782,202101,115,115.000000,115,115
1,249274782,202102,113,114.000000,115,113
2,249274782,202103,188,138.666667,188,113
3,249274782,202104,211,156.750000,211,113
4,249274782,202105,237,187.250000,237,113
5,249274782,202106,188,206.000000,237,188
6,249303228,202101,72,72.000000,72,72
7,249303228,202102,73,72.500000,73,72
8,249303228,202103,80,75.000000,80,72
9,249303228,202104,77,75.500000,80,72


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [16]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_3
0,719374230,202106,4,-3.5
1,719421199,202101,93,NaN
2,719421199,202102,93,0.0
3,719421199,202103,113,10.0
4,719421199,202104,131,13.4
5,719421199,202105,138,15.3
6,719421199,202106,111,0.1
7,719432138,202101,81,NaN
8,719432138,202102,84,3.0
9,719432138,202103,75,-3.0


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [17]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']



Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [18]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)


, regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as ctrx_active_quarter_slope_3
, regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as ctrx_cliente_vip_slope_3
, regr_slope(internet, cliente_antiguedad) over ventana_3 as ctrx_internet_slope_3
, regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as ctrx_cliente_edad_slope_3
, regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as ctrx_cliente_antiguedad_slope_3
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as ctrx_mrentabilidad_slope_3


In [19]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,249302860,202101,134,<NA>,<NA>,<NA>
1,249302860,202102,142,134,8,<NA>
2,249302860,202103,111,142,-31,-23
3,249302860,202104,113,111,2,-29
4,249302860,202105,89,113,-24,-22
5,249302860,202106,103,89,14,-10
6,249363168,202101,89,<NA>,<NA>,<NA>
7,249363168,202102,93,89,4,<NA>
8,249363168,202103,115,93,22,26
9,249363168,202104,115,115,0,22


In [20]:
%%time
%%sql
create or replace table competencia_01 as
SELECT
    * EXCLUDE(clase_ternaria),
    (ctarjeta_debito_transacciones + ctarjeta_visa_transacciones + ctarjeta_master_transacciones) as ctarjeta_total_transacciones,
    (ctarjeta_visa_transacciones + ctarjeta_master_transacciones) as ctarjeta_credito_transacciones,
    if(cpayroll_trx > 0 , 1, 0) as cpayroll,
    (mcaja_ahorro + mcaja_ahorro_adicional + mcaja_ahorro_dolares) as mcaja_ahorro_total,
    (mtarjeta_visa_consumo + mtarjeta_master_consumo) as mtarjeta_credito_consumo,
    (mplazo_fijo_pesos + mplazo_fijo_dolares) as mplazo_fijo_total,
    (Visa_mlimitecompra + Master_mlimitecompra) as credito_mlimitecompra,
    (mcuenta_corriente_adicional+mcuenta_corriente+mcaja_ahorro+mcaja_ahorro_adicional+mcaja_ahorro_dolares+mcuentas_saldo+mplazo_fijo_dolares+mplazo_fijo_pesos+minversion1_pesos+minversion1_dolares+minversion2) as mbanco_total,
    (mprestamos_personales + mprestamos_prendarios + mprestamos_hipotecarios) as mprestamos_total,
    clase_ternaria
FROM competencia_01

CPU times: total: 3.19 s
Wall time: 1.26 s


,Success


In [21]:
%%sql
COPY competencia_01 TO '{{dataset_path}}processed/competencia_01_fe.csv' (FORMAT CSV, HEADER);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
data = pd.read_csv(dataset_path + 'processed/competencia_01_fe.csv')

In [ ]:
exclude_columns = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'tcuentas', 'cdescubierto_preacordado', 'ccaja_seguridad', 'tcallcenter', 'thomebanking', 'tmobile_app', 'Master_status', 'Visa_status']

In [ ]:
campos_data = list(data)[2:-1]
campos_data = [x for x in campos_data if x not in exclude_columns]
print(campos_data)

In [ ]:
nuevos_features = ""
for campo in campos_data:
    nuevos_features += f"\n, lag({campo},1) over ventana_1 as {campo}_lag_1"
    nuevos_features += f"\n, lag({campo},2) over ventana_1 as {campo}_lag_2"
    nuevos_features += f"\n, lag({campo},3) over ventana_1 as {campo}_lag_3"
    nuevos_features += f"\n, ifnull({campo},0) - ifnull({campo}_lag_1,0) as {campo}_delta_1"
    nuevos_features += f"\n, avg({campo}) over ventana_3 as {campo}_avg_3"
    nuevos_features += f"\n, max({campo}) over ventana_3 as {campo}_max_3"
    nuevos_features += f"\n, min({campo}) over ventana_3 as {campo}_min_3"
    nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"

In [ ]:
%%time
%%sql
create or replace table competencia_01_fe as
SELECT
    *
    {{nuevos_features}}
FROM competencia_01
window 
    ventana_1 as (partition by numero_de_cliente order by foto_mes),
    ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row);




Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [ ]:
%%sql
COPY competencia_01_fe TO '{{dataset_path}}processed/competencia_01_fe.csv' (FORMAT CSV, HEADER);